In [1]:
!pip install opendatasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install -q kaggle

In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"dimafrank","key":"e7786e191009a5f2647b768669ed5b0c"}'}

In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [5]:
import opendatasets as od

In [6]:
od.download(
    "https://www.kaggle.com/datasets/vijayuv/onlineretail")

100%|██████████| 7.20M/7.20M [00:00<00:00, 54.5MB/s]

In [7]:
!ls ./onlineretail

OnlineRetail.csv


In [8]:
!pwd

/content


In [9]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 46 kB/s 
     |████████████████████████████████| 199 kB 55.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=92548fe6d23f4836f4a5fcbb7cae1233aa7b3c35e69334fd5e7dca22d4212635
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [10]:
import pyspark as spark
from google.colab import drive
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import array, col, concat_ws, udf, array_remove, size

In [11]:
drive.mount('/content/drive')
!wget --quiet --show-progress https://drive.google.com/drive/u/0/folders/1zKTXVP8ncsRRXgtFxA6MvL-lWirAoc6a

Mounted at /content/drive


In [12]:
SparkSession = SparkSession.builder.appName("Colab").getOrCreate()

In [13]:
df = SparkSession.read.csv('./onlineretail/OnlineRetail.csv',header='true')

In [ ]:
df.show(10)

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|12/1/2010 8:26|     7.65|     17850|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|12/1/2010 8:26|     4.

In [ ]:
df.count()

541909

In [ ]:
df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: string (nullable = true)
 |-- CustomerID: string (nullable = true)
 |-- Country: string (nullable = true)



In [ ]:
df.createOrReplaceTempView("transactions")

In [ ]:
min_support=0.2

query = '''    
         WITH source as (
                        SELECT t.*,
                          CAST(SUBSTRING_INDEX(SUBSTRING_INDEX(InvoiceDate, '/', 2), "/", -1) AS Integer) as day,
                          CAST(SUBSTRING_INDEX(InvoiceDate, '/', 1) AS Integer) as month,                      
                          CAST(SUBSTRING_INDEX(SUBSTRING_INDEX(InvoiceDate, '/', -1), " ", 1) AS Integer) as year                  
                                  
                        FROM transactions as t
                        WHERE Description <> 'null'

                        ),

              total (
                      select count(distinct CustomerId)
                      from source
                    )          


              SELECT *, CNT/ALL_CNT as SUPPORT 
              FROM(
                  SELECT Description as ITEM, count(*) as CNT, (select * from total) as ALL_CNT 
                  FROM source
                  GROUP BY Description
                  ) as tab1
              WHERE (CNT/ALL_CNT) > {min_support}
              ORDER BY SUPPORT DESC

         '''.format(min_support=min_support)

In [ ]:
C1 = SparkSession.sql(query)
C1

58

In [ ]:
C1.createOrReplaceTempView("C1")

In [ ]:
test = SparkSession.sql('''select * from transactions where Description in (select ITEM from C1)''')
test.createOrReplaceTempView("test")
test.show(5)

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6| 12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536373|   85123A|WHITE HANGING HEA...|       6| 12/1/2010 9:02|     2.55|     17850|United Kingdom|
|   536375|   85123A|WHITE HANGING HEA...|       6| 12/1/2010 9:32|     2.55|     17850|United Kingdom|
|   536386|   85099B|JUMBO BAG RED RET...|     100| 12/1/2010 9:57|     1.65|     16029|United Kingdom|
|   536390|   85123A|WHITE HANGING HEA...|      64|12/1/2010 10:19|     2.55|     17511|United Kingdom|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
only showing top 5 rows



In [ ]:
pivotDF = test.groupBy("CustomerID").pivot("Description").count().fillna(value=0)\
              .withColumnRenamed("JUMBO BAG RED RETROSPOT","ITEM1")\
              .withColumnRenamed("REGENCY CAKESTAND 3 TIER","ITEM2")\
              .withColumnRenamed("WHITE HANGING HEART T-LIGHT HOLDER","ITEM3")\
              .withColumnRenamed("CustomerID","TID")
pivotDF.createOrReplaceTempView('pivotDF')             
pivotDF.show(10)

+-----+-----+-----+-----+
|  TID|ITEM1|ITEM2|ITEM3|
+-----+-----+-----+-----+
|17686|    4|    2|    2|
|15555|    0|    3|    6|
|15271|    0|    0|    6|
|13865|    0|    2|    0|
|15574|    0|    2|    0|
|14887|    1|    0|    0|
|16639|    0|    0|    3|
|15052|    0|    0|    1|
|12637|    3|    1|    0|
|13985|   10|    0|    0|
+-----+-----+-----+-----+
only showing top 10 rows



In [ ]:
pivotDF2 = SparkSession.sql('''
                            SELECT *,
                                    CASE 1
                                          WHEN ITEM1>0 AND ITEM2>0 THEN 1
                                          ELSE 0
                                    END AS ITEM1_ITEM2,

                                    CASE 1
                                          WHEN ITEM1>0 AND ITEM3>0 THEN 1
                                          ELSE 0
                                    END AS ITEM1_ITEM3,

                                    CASE 1
                                          WHEN ITEM2>0 AND ITEM3>0 THEN 1
                                          ELSE 0
                                    END AS ITEM2_ITEM3
                    
                            FROM pivotDF
                          '''
                         )


pivotDF2.createOrReplaceTempView('pivotDF2')
pivotDF2.show(5)               

+-----+-----+-----+-----+-----------+-----------+-----------+
|  TID|ITEM1|ITEM2|ITEM3|ITEM1_ITEM2|ITEM1_ITEM3|ITEM2_ITEM3|
+-----+-----+-----+-----+-----------+-----------+-----------+
|17686|    4|    2|    2|          1|          1|          1|
|15555|    0|    3|    6|          0|          0|          1|
|15271|    0|    0|    6|          0|          0|          0|
|13865|    0|    2|    0|          0|          0|          0|
|15574|    0|    2|    0|          0|          0|          0|
+-----+-----+-----+-----+-----------+-----------+-----------+
only showing top 5 rows



In [ ]:
pivotDf3 = SparkSession.sql(
                            '''
                            SELECT "ITEM1_ITEM2" as I, sum(ITEM1_ITEM2) as freq
                            FROM pivotDF2
                            UNION ALL
                            SELECT "ITEM1_ITEM3" as I, sum(ITEM1_ITEM3) as freq
                            FROM pivotDF2
                            UNION ALL
                            SELECT "ITEM2_ITEM3" as I, sum(ITEM2_ITEM3) as freq
                            FROM pivotDF2

                            '''
                           ).show(5)

+-----------+----+
|          I|freq|
+-----------+----+
|ITEM1_ITEM2| 183|
|ITEM1_ITEM3| 223|
|ITEM2_ITEM3| 263|
+-----------+----+



In [ ]:
test1 =  SparkSession.sql('''select * from transactions where CustomerID='17686' ''')
test1.createOrReplaceTempView("test1")

test1.show(10)

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|   543555|    22193|RED DINER WALL CLOCK|       2|2/10/2011 10:18|      8.5|     17686|United Kingdom|
|   543555|    22192|BLUE DINER WALL C...|       2|2/10/2011 10:18|      8.5|     17686|United Kingdom|
|   543555|    22191|IVORY DINER WALL ...|       2|2/10/2011 10:18|      8.5|     17686|United Kingdom|
|   543555|    22663|JUMBO BAG DOLLY G...|      10|2/10/2011 10:18|     1.95|     17686|United Kingdom|
|   543555|    22411|JUMBO SHOPPER VIN...|      10|2/10/2011 10:18|     1.95|     17686|United Kingdom|
|   543555|    21930|JUMBO STORAGE BAG...|      10|2/10/2011 10:18|     1.95|     17686|United Kingdom|
|   543555|    22386|JUMBO BAG PINK PO...|      10|2/10/2011 10:

In [59]:
class UserSessionBuilder:
 
    def __init__(self, datafile, items, transactions, support=0.4, confidence=0.8):
        # datafile      --> Expects to get PySpark DataFrame
        # items         --> Name of a column that holds Items in a given dataset
        # transactions  --> Name of a column that holds Transaction ID in a given dataset
        # support       --> Default Treshold is 0.2
        # confidence    --> Required confidence level (Default is 0.8)

        self.datafile=datafile
        self.items=items
        self.transactions=transactions
        self.support=support
        self.confidence=confidence


    def create_support(self):
        self.datafile.createOrReplaceTempView("temp")
        query = '''
                WITH SOURCE as (
                                SELECT d.*, REPLACE({ITEMS}, ' ', '_') as ITEMS                                                  
                                FROM {data} as d
                                WHERE {ITEMS} <> 'null'
                                ),

                    TOTAL as ( select count(distinct ITEMS) from SOURCE ),          

                    SUPPORT_ITEM_SET1 (
                          SELECT *, CNT/ALL_CNT as SUPPORT 
                          FROM(
                              SELECT ITEMS, count(*) as CNT, (select * from TOTAL) as ALL_CNT 
                              FROM SOURCE
                              GROUP BY ITEMS
                              ) as tab1
                          WHERE (CNT/ALL_CNT) > {min_support}
                          ORDER BY SUPPORT DESC
                                      )
                                    
                    SELECT *
                    FROM SOURCE
                    WHERE ITEMS in (SELECT ITEMS FROM SUPPORT_ITEM_SET1)

              '''.format(data='temp', ITEMS=self.items, min_support=self.support)

        SparkSession.sql(query).createOrReplaceTempView("support")
        return SparkSession.sql(query)


    def create_basket(self):
        global item_mapping 
        basket = self.create_support()
        basket = basket.groupBy('{}'.format(self.transactions)).pivot('{}'.format(self.items)).count().fillna(value=0)
        new_col_names = ['ITEM{i}'.format(i=i) if i>=1 else self.transactions for i in range(len(basket.schema.names))] 
        item_mapping = {new_col_names[i]: basket.schema.names[i] for i in range(len(basket.schema.names))}
        basket = basket.toDF(*new_col_names)
        basket = basket.select("{}".format(self.transactions), array(*new_col_names[1:]).alias("Basket")) 
        basket = basket.withColumn(
                "BasketBinaryVector",
                   spark.sql.functions.expr("""transform(
                              Basket, 
                              (x, i) -> CASE WHEN x > 0 THEN 1
                                            ELSE x
                                        END
                          )
                      """)
                  )
        
        basket = basket.withColumn(
                "BasketVector",
                   spark.sql.functions.expr("""transform(
                              Basket, 
                              (x, i) -> CASE WHEN x > 0 THEN concat("ITEM",i+1)
                                            ELSE ""
                                        END
                          )
                      """)
                  )
        
        basket = basket.select(self.transactions, "Basket", "BasketBinaryVector" ,array_remove('BasketVector',""))
        basket = basket.withColumn("size", size(col("array_remove(BasketVector, )"))).withColumnRenamed("array_remove(BasketVector, )", "BasketClean")
        basket.createOrReplaceTempView("basket")
        return basket


    def create_item_sets(self):
        global max_item_set
        basket = self.create_basket()
        max_item_set = SparkSession.sql("SELECT MAX(size) FROM basket").collect()[0][0]
        i=1
        while i<=max_item_set:         
          temp = SparkSession.sql('''
                                  SELECT BasketClean, count(*) as CNT
                                  FROM basket 
                                  WHERE size == {SIZE}
                                  GROUP BY BasketClean   
                                  ORDER BY CNT DESC   
                                  '''.format(SIZE=i))

          temp.createOrReplaceTempView("item_set_{n}".format(n=i))  
          temp.show(5)                     
          i+=1
               


    def calculate_confidence(self):
          return None




    def get_mapping(self):
       self.create_basket()
       del item_mapping[self.transactions]
       return item_mapping



In [47]:
UserSessionBuilder(df, 'Description', 'CustomerID').create_support().show(10)

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+--------------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|               ITEMS|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+--------------------+
|   536365|   85123A|WHITE HANGING HEA...|       6| 12/1/2010 8:26|     2.55|     17850|United Kingdom|WHITE_HANGING_HEA...|
|   536373|   85123A|WHITE HANGING HEA...|       6| 12/1/2010 9:02|     2.55|     17850|United Kingdom|WHITE_HANGING_HEA...|
|   536375|   85123A|WHITE HANGING HEA...|       6| 12/1/2010 9:32|     2.55|     17850|United Kingdom|WHITE_HANGING_HEA...|
|   536386|   85099B|JUMBO BAG RED RET...|     100| 12/1/2010 9:57|     1.65|     16029|United Kingdom|JUMBO_BAG_RED_RET...|
|   536390|   85123A|WHITE HANGING HEA...|      64|12/1/2010 10:19|     2.55|     17511|United Kingdom|WHITE_HANGING_HEA...|


In [48]:
UserSessionBuilder(df, 'Description', 'CustomerID').create_basket().show(10)

+----------+------------+------------------+--------------------+----+
|CustomerID|      Basket|BasketBinaryVector|         BasketClean|size|
+----------+------------+------------------+--------------------+----+
|     17686|[4, 2, 2, 2]|      [1, 1, 1, 1]|[ITEM1, ITEM2, IT...|   4|
|     13772|[0, 2, 0, 0]|      [0, 1, 0, 0]|             [ITEM2]|   1|
|     15555|[0, 2, 3, 6]|      [0, 1, 1, 1]|[ITEM2, ITEM3, IT...|   3|
|     15271|[0, 1, 0, 6]|      [0, 1, 0, 1]|      [ITEM2, ITEM4]|   2|
|     13865|[0, 1, 2, 0]|      [0, 1, 1, 0]|      [ITEM2, ITEM3]|   2|
|     15574|[0, 1, 2, 0]|      [0, 1, 1, 0]|      [ITEM2, ITEM3]|   2|
|     13282|[0, 2, 0, 0]|      [0, 1, 0, 0]|             [ITEM2]|   1|
|     13610|[0, 1, 0, 0]|      [0, 1, 0, 0]|             [ITEM2]|   1|
|     14887|[1, 0, 0, 0]|      [1, 0, 0, 0]|             [ITEM1]|   1|
|     14204|[0, 1, 0, 0]|      [0, 1, 0, 0]|             [ITEM2]|   1|
+----------+------------+------------------+--------------------+----+
only s

In [60]:
UserSessionBuilder(df, 'Description', 'CustomerID').create_item_sets()

+-----------+---+
|BasketClean|CNT|
+-----------+---+
|    [ITEM3]|411|
|    [ITEM4]|372|
|    [ITEM1]|272|
|    [ITEM2]|248|
+-----------+---+

+--------------+---+
|   BasketClean|CNT|
+--------------+---+
|[ITEM2, ITEM3]|119|
|[ITEM3, ITEM4]|102|
|[ITEM2, ITEM4]| 89|
|[ITEM1, ITEM4]| 88|
|[ITEM1, ITEM3]| 56|
+--------------+---+
only showing top 5 rows

+--------------------+---+
|         BasketClean|CNT|
+--------------------+---+
|[ITEM2, ITEM3, IT...| 73|
|[ITEM1, ITEM2, IT...| 47|
|[ITEM1, ITEM3, IT...| 41|
|[ITEM1, ITEM2, IT...| 39|
+--------------------+---+

+--------------------+---+
|         BasketClean|CNT|
+--------------------+---+
|[ITEM1, ITEM2, IT...| 47|
+--------------------+---+



In [49]:
UserSessionBuilder(df, 'Description', 'CustomerID').get_mapping()

{'ITEM1': 'JUMBO BAG RED RETROSPOT',
 'ITEM2': 'PARTY BUNTING',
 'ITEM3': 'REGENCY CAKESTAND 3 TIER',
 'ITEM4': 'WHITE HANGING HEART T-LIGHT HOLDER'}

In [ ]:
175/530

0.330188679245283

In [ ]:
135/319

0.4231974921630094

In [ ]:
95/319

0.29780564263322884

In [ ]:
SparkSession = SparkSession.builder.appName("Colab").config("spark.memory.offHeap.size","10g").getOrCreate()

In [ ]:
df = SparkSession.read.csv('./onlineretail/OnlineRetail.csv',header='true')

In [ ]:
df.show(5)

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
only showing top 5 rows



In [ ]:
df.createOrReplaceTempView("df")
query = '''
          SELECT *, REPLACE(Description, ' ', '_') as ITEMS                                                  
          FROM df 
          WHERE Description <> 'null'
       '''
df2 = SparkSession.sql(query)
df2.show(5)

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+--------------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|               ITEMS|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+--------------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|WHITE_HANGING_HEA...|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom| WHITE_METAL_LANTERN|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|CREAM_CUPID_HEART...|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|KNITTED_UNION_FLA...|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|RED_WOOLLY_HOTTIE...|
+-------

In [ ]:
test=df2.groupBy('CustomerID').pivot('ITEMS').count().fillna(value=0)
test.show(5)

+----------+---------------------------------+-------------------------------------+--------------------------------------+-------------------------------------+--------------------------------------+--------------------------------------+--------------------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+--

In [ ]:
len(test.schema.names)

4224

In [ ]:
new_col_names = ['ITEM{i}'.format(i=i) if i>=1 else 'CustomerID' for i in range(len(test.schema.names))] 
new_col_names[:10]

['CustomerID',
 'ITEM1',
 'ITEM2',
 'ITEM3',
 'ITEM4',
 'ITEM5',
 'ITEM6',
 'ITEM7',
 'ITEM8',
 'ITEM9']

In [ ]:
len(new_col_names)

4224

In [ ]:
item_mapping = {new_col_names[i]: test.schema.names[i] for i in range(len(test.schema.names))}

In [ ]:
print(str(item_mapping))

{'CustomerID': 'CustomerID', 'ITEM1': '"ASSORTED_FLOWER_COLOUR_""LEIS"""', 'ITEM2': '"CHARLIE+LOLA""EXTREMELY_BUSY""_SIGN"', 'ITEM3': '"FLOWER_GLASS_GARLAND_NECKL.36""BLACK"', 'ITEM4': '"FLOWER_GLASS_GARLAND_NECKL.36""BLUE"', 'ITEM5': '"FLOWER_GLASS_GARLAND_NECKL.36""GREEN"', 'ITEM6': '"FLOWER_GLASS_GARLD_NECKL36""AMETHYST"', 'ITEM7': '"FLOWER_GLASS_GARLD_NECKL36""TURQUOIS"', 'ITEM8': '"LETTER_""A""_BLING_KEY_RING"', 'ITEM9': '"LETTER_""B""_BLING_KEY_RING"', 'ITEM10': '"LETTER_""C""_BLING_KEY_RING"', 'ITEM11': '"LETTER_""D""_BLING_KEY_RING"', 'ITEM12': '"LETTER_""E""_BLING_KEY_RING"', 'ITEM13': '"LETTER_""F""_BLING_KEY_RING"', 'ITEM14': '"LETTER_""G""_BLING_KEY_RING"', 'ITEM15': '"LETTER_""H""_BLING_KEY_RING"', 'ITEM16': '"LETTER_""I""_BLING_KEY_RING"', 'ITEM17': '"LETTER_""J""_BLING_KEY_RING"', 'ITEM18': '"LETTER_""K""_BLING_KEY_RING"', 'ITEM19': '"LETTER_""L""_BLING_KEY_RING"', 'ITEM20': '"LETTER_""M""_BLING_KEY_RING"', 'ITEM21': '"LETTER_""N""_BLING_KEY_RING"', 'ITEM22': '"LETTER_""

In [ ]:
temp = test.toDF(*new_col_names)
temp.show(5)

+----------+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+

In [ ]:
from pyspark.sql.functions import array, col

temp.select("CustomerID", array(*new_col_names[1:]).alias("preferences")).show(truncate=False)

Py4JJavaError: ignored